In [1]:
import os
import sys
sys.path.insert(0, "../")

In [2]:
from core.nl_parser import SYSTEM, USER, NLParser

# Setup

In [ ]:
open_ai_key = os.environ["OPENAI_API_KEY"]
antropic_key = os.environ["ANTHROPIC_API_KEY"]


KeyError: 'ANTHROPIC_API_KEY'

In [6]:
# nl_parser = NLParser(open_ai_key)

In [ ]:
# !pip install \
#   "pydantic>=2.7,<3" \
#   "langchain>=0.3,<0.4" \
#   "langchain-core>=0.3,<0.4" \
#   "langchain-community>=0.3,<0.4" \
#   "langchain-openai>=0.2,<0.3" \
#   "openai>=1.40"

# Generate

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [9]:
ENDPOINT="us-central1-aiplatform.googleapis.com"
REGION="us-central1"
PROJECT_ID="support-robot-448808"
BUCKET_NAME="base_bucket-2025"
BUCKET_URI = f"gs://{BUCKET_NAME}"
MODEL_LOCATION = "us-central1"

In [10]:
from langchain_google_vertexai import ChatVertexAI
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION) 

In [11]:
duration = 90
focus = "Endurance"
prefs = ""

In [17]:
model = "gpt-5-mini"
llm_open_ai = ChatOpenAI(model=model, temperature=1, api_key=open_ai_key)

In [ ]:
# Anthropic removed - using Google Vertex AI instead

In [12]:
model = "gemini-2.5-flash"
llm_vertex = ChatVertexAI(model=model, temperature=1.0)

In [13]:
prompt = ChatPromptTemplate.from_messages([("system", SYSTEM), ("user", USER)])

In [14]:
%%time
msg = prompt.format_messages(duration_min=duration, focus=focus, prefs=prefs or "—")
raw = llm_open_ai.invoke(msg)
print(raw)

CPU times: user 690 μs, sys: 31 μs, total: 721 μs
Wall time: 734 μs


NameError: name 'llm_open_ai' is not defined

In [ ]:
# Anthropic test removed - using Google Vertex AI instead

content='```json\n{\n  "name": "Steady State Endurance",\n  "focus": "Endurance",\n  "description": "A classic aerobic endurance session designed to build mitochondrial density and fat oxidation capacity. The sustained Zone 2 effort develops the aerobic base crucial for long rides and recovery between high-intensity blocks. Perfect for building durability without creating excessive fatigue.",\n  "steps": [\n    {"duration_s": 300, "pct_ftp": 0.50, "pct_ftp_end": 0.60, "kind": "warmup", "note": "Easy spin to wake up the legs and prepare joints and muscles"},\n    {"duration_s": 300, "pct_ftp": 0.60, "pct_ftp_end": 0.68, "kind": "warmup", "note": "Gradually increase effort into Zone 2, focusing on smooth pedaling"},\n    {"duration_s": 240, "pct_ftp": 0.68, "pct_ftp_end": 0.72, "kind": "warmup", "note": "Final warmup ramp to settle into comfortable Zone 2 rhythm"},\n    {"duration_s": 1200, "pct_ftp": 0.72, "kind": "steady", "note": "First main block—find your sustainable aerobic pace, c

In [47]:
%%time
msg = prompt.format_messages(duration_min=duration, focus=focus, prefs=prefs or "—")
raw = llm_vertex.invoke(msg)
print(raw)

content='```json\n{\n "name": "90-Min Foundational Endurance",\n "focus": "Endurance",\n "description": "This session builds aerobic capacity and fat-burning efficiency by focusing on sustained Zone 2 efforts. Consistent power output in this zone enhances mitochondrial function and capillary density, crucial for improving long-distance performance without excessive fatigue.",\n "steps": [\n  {\n   "duration_s": 720,\n   "pct_ftp": 0.50,\n   "kind": "warmup",\n   "pct_ftp_end": 0.72,\n   "note": "Progressive ramp to gently elevate heart rate and muscle temperature, preparing the aerobic system for sustained work."\n  },\n  {\n   "duration_s": 1800,\n   "pct_ftp": 0.72,\n   "kind": "steady",\n   "note": "First sustained block in Zone 2. Focus on a steady, consistent power output and smooth pedaling."\n  },\n  {\n   "duration_s": 300,\n   "pct_ftp": 0.55,\n   "kind": "steady",\n   "note": "Active recovery to slightly lower heart rate and clear metabolites, preparing for the next effort."\

In [50]:
def get_costs(raw_response, pricing_mapping, model_id="gemini-2.5-flash"):
    costs = 0
    model_pricing_mapping = pricing_mapping[model_id]
    input_token_price = model_pricing_mapping["input_tokens"] / 10 ** 6
    output_token_price = model_pricing_mapping["output_tokens"] / 10 ** 6
    
    usage = raw_response.response_metadata["usage_metadata"]
    costs += usage["prompt_token_count"] * input_token_price
    costs += usage["thoughts_token_count"] * output_token_price
    costs += usage["candidates_token_count"] * output_token_price
    return costs

In [62]:
# In $ / 1 Mio Tokens
pricing_mapping = {
    "gemini-2.5-flash": {
        "input_tokens": 0.3,
        "output_tokens": 2.5
    },
    "gemini-2.5-flash-lite": {
        "input_tokens": 0.1,
        "output_tokens": 0.4
    },
    "gemini-2.5-pro": {
        "input_tokens": 1.25,
        "output_tokens": 10
    }
}

In [66]:
model = "gemini-2.5-flash-lite"

llm_vertex = ChatVertexAI(model=model, temperature=1.0)

In [67]:
%%time
msg = prompt.format_messages(duration_min=duration, focus=focus, prefs=prefs or "—")
raw = llm_vertex.invoke(msg)

CPU times: user 5.79 ms, sys: 3.51 ms, total: 9.29 ms
Wall time: 1.98 s


In [68]:
get_costs(raw, pricing_mapping, model)

0.00031860000000000005